In [ ]:
# set IO divisions
IO_left, IO_x_mid, IO_right =  568, 615, 678
IO_front, IO_z_mid, IO_back = 1128, 1185, 1279

# rostral medial
q1_minX, q1_maxX = IO_left, IO_x_mid
q1_minZ, q1_maxZ = IO_front, IO_z_mid

# rostral lateral
q2_minX, q2_maxX = IO_x_mid, IO_right
q2_minZ, q2_maxZ = IO_front, IO_z_mid

# caudal medial
q3_minX, q3_maxX = IO_left, IO_x_mid
q3_minZ, q3_maxZ = IO_z_mid, IO_back

# caudal lateral
q4_minX, q4_maxX = IO_x_mid, IO_right
q4_minZ, q4_maxZ = IO_z_mid, IO_back

In [ ]:
def total_along_axis(channel, dataset, axis, binsize, atlas_res=10):
    parent, children = bt.children_from(83, depth=0)
    areas = [parent] + children
    points = np.array(bt._get_cells_in(areas, dataset, channel)).T
    x_bins, y_bins, z_bins = btp.get_bins(0, binsize), btp.get_bins(1, binsize), btp.get_bins(2, binsize)
    hist, _ = np.histogramdd(points, bins=(x_bins, y_bins, z_bins), range=((0,1140),(0,800),(0,1320)), normed=False)
    if hist.sum() != 0:
        hist = hist / hist.sum() # turn into probability density distribution
    else:
        print('Warning: did not perform conversion to probability density')
    hist = np.sum(hist, axis=axis) # take the maximum projection of the distribution
    av_im = hist if axis == 0 else hist.T # side-on orientation does not need axis swapping
    tot_along_axis = np.sum(av_im, axis=0) # condense onto axis for splitting
    return tot_along_axis

In [ ]:
# no longer needed
def num_in_quarter(minX, maxX, minZ, maxZ):
    cells_x, _, cells_z = bt._get_cells_in(83, d, channel='r', left=None)
    x_filtered_idxs = [i for i, x in enumerate(cells_x) if x > minX and x < maxX]
    z_filtered_idxs = [i for i, z in enumerate(cells_z) if z > minZ and z < maxZ]
    num_in_bounds = len([c for i, c in enumerate(cells_x) if i in x_filtered_idxs and i in z_filtered_idxs])
    return num_in_bounds

area_labels = ['IO, rostral medial', 'IO, rostral lateral', 'IO, caudal medial', 'IO, caudal lateral']
num_cells_in_each_quarter = []
for d in bt.datasets:
    q1_num = num_in_quarter(q1_minX, q1_maxX, q1_minZ, q1_maxZ)
    q2_num = num_in_quarter(q2_minX, q2_maxX, q2_minZ, q2_maxZ)
    q3_num = num_in_quarter(q3_minX, q3_maxX, q3_minZ, q3_maxZ)
    q4_num = num_in_quarter(q4_minX, q4_maxX, q4_minZ, q4_maxZ)
    num_cells_in_each_quarter.append([q1_num, q2_num, q3_num, q4_num])
print(num_cells_in_each_quarter)

In [ ]:
def num_in_quarter(axis, split, dataset):
        cells_along_axis = total_along_axis('r', dataset, axis, 50) # 2 is along x axis
        cells_in_side1 = np.sum(cells_along_axis[:split])
        cells_in_side2 = np.sum(cells_along_axis[split:])
        return cells_in_side1, cells_in_side2